In [2]:
import numpy as np 
from tqdm import tqdm_notebook as tqdm

In [9]:
class AlternatingLeastSquares():
    def __init__(self, R, k, reg_param, epochs, verbose = False):

        self._R = R
        self._k = k 
        self._num_users, self._num_items = R.shape
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose 


    def fit(self):
        self._users = np.random.normal(size = (self._num_users, self._k))
        self._items = np.random.normal(size = (self._num_items, self._k))

        # train while epochs 
        self._training_process = []
        self._user_error = 0; self._item_error = 0;
        for epoch in range(self._epochs):
            for i, Ri in enumerate(self._R):
                self._users[i] = self.user_latent(i, Ri)
                self._user_error = self.cost()

            for j, Rj in enumerate(self._R.T):
                self._items[j] = self.item_latent(j, Rj)
                self._item_error = self.cost()

            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0) :
                print('Iteration %d  cost = %.4f' % (epoch + 1, cost))


    def cost(self):

        xi, yi = self._R.nonzero()
        cost = 0 
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost / len(xi))


    def user_latent(self, i, Ri):

        du = np.linalg.solve(np.dot(self._items.T, np.dot(np.diag(Ri), self._items)) + self._reg_param * np.eye(self._k),
        np.dot(self._items.T, np.dot(np.diag(Ri), self._R[i].T))).T 
        return du 


    def item_latent(self, j, Rj):
        
        di = np.linalg.solve(np.dot(self._users.T, np.dot(np.diag(Rj), self._users)) + self._reg_param * np.eye(self._k),
        np.dot(self._users.T, np.dot(np.diag(Rj), self._R[:,j])))
        return di 


    def get_prediction(self, i, j):
        return self._users[i, :].dot(self._items[j, :].T)


    def get_complete_matrix(self):
        return self._users.dot(self._items.T)



if __name__ == '__main__' :
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0]
    ])


In [10]:
als = AlternatingLeastSquares(R = R, k = 3, reg_param=0.01, epochs=100, verbose=True)
als.fit()

Iteration 10  cost = 0.0241
Iteration 20  cost = 0.0136
Iteration 30  cost = 0.0109
Iteration 40  cost = 0.0093
Iteration 50  cost = 0.0081
Iteration 60  cost = 0.0072
Iteration 70  cost = 0.0065
Iteration 80  cost = 0.0059
Iteration 90  cost = 0.0054
Iteration 100  cost = 0.0050
